
 This code generates a unique name for each DICOM image
 This code convers DICOM files to JPG file
 This code is designed to handle a deeper folder structure
 The os.walk() to recursively traverse through all folders and subfolders, locating DICOM files and converting them to JPG, extract them and combine them in the new output folder.  

In [1]:
pip install pydicom Pillow

Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
from pathlib import Path
import pydicom
from PIL import Image
import matplotlib.pyplot as plt
from pydicom.pixel_data_handlers.util import apply_modality_lut

In [ ]:
def convert_dicom_to_jpg(dicom_path, output_folder):
    # Load DICOM file
    dicom_data = pydicom.dcmread(dicom_path)

    # Ensure pixel data is monochrome
    if dicom_data.PhotometricInterpretation != "MONOCHROME2":
        raise ValueError("Only MONOCHROME2 images are supported.")

    # Apply modality LUT
    pixel_data = apply_modality_lut(dicom_data.pixel_array, dicom_data)

    # Select a middle slice if the pixel data has three dimensions
    if len(pixel_data.shape) == 3:
        middle_slice = pixel_data.shape[0] // 2
        pixel_data = pixel_data[middle_slice, :, :]

    # Create output file path with a new name (e.g., "brain_tumor_001.jpg")
    jpg_filename = f"brain_tumor_{dicom_data.SOPInstanceUID}.jpg"
    jpg_path = os.path.join(output_folder, jpg_filename)

    # Plot and save the DICOM pixel array using matplotlib
    plt.imshow(pixel_data, cmap=plt.cm.bone)
    plt.axis('off')  # Turn off axis labels
    plt.savefig(jpg_path, bbox_inches='tight', pad_inches=0, format='jpg')
    plt.close()


def process_dicom_folder(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Recursively iterate over all files in the input folder and its subfolders
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(".dcm"):
                dicom_path = os.path.join(root, file)
                convert_dicom_to_jpg(dicom_path, output_folder)

if __name__ == "__main__":
    # Replace 'input_folder' with the path to the root folder containing DICOM files
    input_folder = r"C:\Users\net pc\Desktop\THESIS PROJECT\Test Dataset\BRAIN TUMOR TEST (manifest-1695134609823\ReMIND.py"

    # Replace 'output_folder' with the path where you want to save the converted JPG images
    output_folder = r"C:\Users\net pc\Desktop\THESIS PROJECT\Test Dataset\BRAIN TUMOR TEST (manifest-1695134609823\combined_brain_test.py"

    process_dicom_folder(input_folder, output_folder)